In [143]:
from sklearn.preprocessing import PolynomialFeatures
import sklearn
import pandas as pd
import numpy as np
import os
import pycountry #pycountry provides the ISO databases for the standards:

In [50]:
#change working directory to 5102 project folder
os.chdir(r"/Users/chung/Google Drive/2018R1 Regression in Practice (STAT5102)/5012 Project")

In [75]:
#read project_data.sas7bdat
world_bank = pd.read_sas(r"project_data.sas7bdat", encoding="utf-8")

In [139]:
world_bank

,Country Name,Country Code,Continent,Adjusted savings: education expe,Adolescent fertility rate (birth,Age dependency ratio (% of worki,"Age dependency ratio, young (% o",Agricultural land (% of land are,"Birth rate, crude (per 1,000 peo","Fertility rate, total (births pe",...,"Life expectancy at birth, female","Life expectancy at birth, male (","Life expectancy at birth, total","Mortality rate, infant (per 1,00","Mortality rate, under-5 (per 1,0","Mortality rate, under-5, female","Mortality rate, under-5, male (p",Population growth (annual %),"Immunization, DPT (% of children","Immunization, measles (% of chil"
0,Afghanistan,AFG,AS,NaN,99.095600,103.119626,98.722026,58.123668,37.952000,5.659,...,60.877,58.384,59.600098,74.3,103.7,100.4,106.7,2.458417,66.0,62.0
1,Albania,ALB,EU,2.842804,16.994600,49.098446,34.036210,43.843066,12.607000,1.741,...,80.149,73.959,76.978512,16.1,18.1,16.3,19.7,-0.033072,99.0,99.0
2,Algeria,DZA,AF,4.467196,9.894599,46.741610,39.796430,17.370914,24.514000,2.817,...,72.229,69.081,70.616610,18.4,21.5,19.0,23.8,1.850438,95.0,95.0
3,American Samoa,ASM,OC,NaN,NaN,NaN,NaN,24.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.088667,NaN,NaN
4,Andorra,ADO,999,2.889150,NaN,NaN,NaN,42.765957,9.800000,1.220,...,NaN,NaN,NaN,2.9,3.5,3.1,3.8,-0.960625,99.0,99.0
5,Angola,AGO,AF,3.516770,178.994000,100.915476,96.050884,46.835646,46.327000,6.218,...,52.138,49.241,50.654171,104.6,173.1,164.5,181.1,3.235950,91.0,93.0
6,Antigua and Barbuda,ATG,NA,2.159410,51.755000,50.300659,39.428660,20.454545,16.966000,2.130,...,77.795,72.990,75.333902,9.8,10.6,9.4,11.7,1.075310,98.0,98.0
7,Argentina,ARG,SA,6.009790,55.372400,55.033758,38.566710,53.611114,17.188000,2.215,...,79.462,72.046,75.663561,13.3,14.8,13.3,16.3,0.872126,94.0,98.0
8,Armenia,ARM,AS,2.219690,27.481600,45.059361,29.758869,60.919944,14.140000,1.738,...,77.657,70.946,74.219659,16.2,18.2,16.1,20.2,-0.157056,94.0,97.0
9,Aruba,ABW,NA,5.519760,29.294400,45.157233,30.206740,11.111111,10.804000,1.701,...,77.429,72.593,74.952024,NaN,NaN,NaN,NaN,0.176342,NaN,NaN


In [ ]:
#Drop the column Year and YearCode
world_bank = world_bank.drop(["Year", "YearCode"], axis=1)

In [ ]:
nrow = world_bank.shape[0]
na_portion = world_bank.isna().sum() / nrow

#remove columns with more than 20% Nan
world_bank = world_bank.loc[:, na_portion < .2]

In [ ]:
#import functions for converting country code into continents codes

from pycountry_convert import (
    map_countries,
    country_alpha2_to_country_name,
    country_name_to_country_alpha2,
    country_alpha3_to_country_alpha2,

    country_alpha2_to_continent_code,

    COUNTRY_NAME_FORMAT_UPPER
)

In [130]:
#convert country code into continents codes

continents = []
fail_codes = []
for country_code in world_bank["Country Code"]:
    try:
        alpha2 = country_alpha3_to_country_alpha2(country_code)
        continent = country_alpha2_to_continent_code(alpha2)
    except KeyError:
        fail_codes.append(country_code)
        continents.append("999")
    else:    
        
        continents.append(continent)

In [140]:
#see what's in the continents list variable
set(continents)

{'999', 'AF', 'AS', 'EU', 'NA', 'OC', 'SA'}

In [133]:
#insert continents into the world_bank dataframe
world_bank.insert(2, "Continent", continents)

In [132]:
#see which country code failed to get continent code
fail_codes

['ADO', 'CHI', 'ZAR', 'IMY', 'KSV', 'ROM', 'SXM', 'TMP', 'WBG']

In [129]:
#will manually impute these codes in excel later
world_bank.loc[world_bank['Country Code'].isin(fail_codes)]

,Country Name,Country Code,Adjusted savings: education expe,Adolescent fertility rate (birth,Age dependency ratio (% of worki,"Age dependency ratio, young (% o",Agricultural land (% of land are,"Birth rate, crude (per 1,000 peo","Fertility rate, total (births pe",GDP (constant 2005 US$),...,"Life expectancy at birth, female","Life expectancy at birth, male (","Life expectancy at birth, total","Mortality rate, infant (per 1,00","Mortality rate, under-5 (per 1,0","Mortality rate, under-5, female","Mortality rate, under-5, male (p",Population growth (annual %),"Immunization, DPT (% of children","Immunization, measles (% of chil"
4,Andorra,ADO,2.88915,NaN,NaN,NaN,42.765957,9.800,1.2200,NaN,...,NaN,NaN,NaN,2.9,3.5,3.1,3.8,-0.960625,99.0,99.0
38,Channel Islands,CHI,NaN,8.805599,45.139071,22.181480,47.894737,9.735,1.4360,NaN,...,81.974,77.791,79.831488,NaN,NaN,NaN,NaN,0.599216,NaN,NaN
43,"Congo, Dem. Rep.",ZAR,2.42834,133.980800,93.408939,87.932098,11.362784,44.017,6.2510,9.438752e+09,...,50.744,47.315,48.987683,104.5,155.1,146.0,163.7,2.779639,60.0,74.0
91,Isle of Man,IMY,NaN,NaN,NaN,NaN,76.140351,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.835704,NaN,NaN
102,Kosovo,KSV,NaN,NaN,51.880342,41.709402,NaN,18.530,2.2893,4.776328e+09,...,72.010,67.860,69.884390,NaN,NaN,NaN,NaN,0.803249,NaN,NaN
156,Romania,ROM,3.35741,30.927000,42.552628,21.402773,61.534449,9.900,1.3300,1.121886e+11,...,77.300,69.800,73.458537,11.8,13.5,12.0,15.0,-0.197584,94.0,95.0
168,Sint Maarten (Dutch part),SXM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.333512,NaN,NaN
190,Timor-Leste,TMP,3.27329,57.646600,101.670610,95.441978,24.546066,36.445,5.5780,7.328473e+08,...,67.500,64.463,65.944463,51.5,61.7,56.4,66.6,2.879662,72.0,66.0
210,West Bank and Gaza,WBG,NaN,47.523600,81.274952,76.263474,41.262458,31.237,4.2160,NaN,...,74.342,71.020,72.640488,20.2,23.9,21.8,25.8,2.898629,NaN,NaN


In [137]:
#export world_bank to excel file to share with groupmates (18 Nov 2018)
writer = pd.ExcelWriter('world_bank.xlsx')
world_bank.to_excel(writer, 'Sheet1')
writer.save()

In [ ]:
#Want to use the PolynomialFeatures function to generate quadratic terms for Polynomial regression

In [141]:
#test PolynomialFeatures function
data = pd.DataFrame.from_dict({
    'x': np.random.randint(low=1, high=10, size=5),
    'y': np.random.randint(low=-1, high=1, size=5),
})

p = PolynomialFeatures(degree=2).fit(data)
p.get_feature_names(data.columns)

['1', 'x', 'y', 'x^2', 'x y', 'y^2']

In [142]:
#have to deal with continent column.
quadratic_terms = PolynomialFeatures(degree=2).fit(world_bank)

ValueError: could not convert string to float: 'AF'